# IBM Applied Data Science Capstone Course by Coursera
### Week 3 Part 1
- Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.
***
### 1. Import libraries

In [1]:
pip install folium

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrape data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"e4f1200f-d7b0-49c1-b49f-f4b1c207d59c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario"

**Using BeautifulSoup**

```python
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
```

In [5]:
# append the data into the respective lists
table_contents=[]
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        #postalCodeList.append(cells[0].join(soup.find('span')['data-value']))
        #boroughList.append(cells[1].text)
        #neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
        cell = {}
        if row.span.text=='Not assigned':
            pass
        else:
            #postalCodeList.append(row.p.text[:3])
            #boroughList.append((row.span.text).split('(')[0])
            #neighborhoodList.append((((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' '))
            cell['PostalCode'] = row.p.text[:3]
            cell['Borough'] = (row.span.text).split('(')[0]
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            table_contents.append(cell)

table_contents


[{'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M1C',
  'Borough': 'Scarborough',
  'Neighborhood': 'Rouge Hill, Port Union, Highland Creek'},
 {'PostalCode': 'M1E',
  'Borough': 'Scarborough',
  'Neighborhood': 'Guildwood, Morningside, West Hill'},
 {'PostalCode': 'M1G', 'Borough': 'Scarborough', 'Neighborhood': 'Woburn'},
 {'PostalCode': 'M1H', 'Borough': 'Scarborough', 'Neighborhood': 'Cedarbrae'},
 {'PostalCode': 'M1J',
  'Borough': 'Scarborough',
  'Neighborhood': 'Scarborough Village'},
 {'PostalCode': 'M1K',
  'Borough': 'Scarborough',
  'Neighborhood': 'Kennedy Park, Ionview, East Birchmount Park'},
 {'PostalCode': 'M1L',
  'Borough': 'Scarborough',
  'Neighborhood': 'Golden Mile, Clairlea, Oakridge'},
 {'PostalCode': 'M1M',
  'Borough': 'Scarborough',
  'Neighborhood': 'Cliffside, Cliffcrest, Scarborough Village West'},
 {'PostalCode': 'M1N',
  'Borough': 'Scarborough',
  'Neighborhood': 'Birch Cliff, Cliffside West'},
 

In [6]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame(table_contents)

#the below has been done as per the direction given from discussion forum
#toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
#                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
#                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
#                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 3. Drop cells with a borough that is "Not assigned"

In [7]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 4. Group neighborhoods in the same borough


In [8]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 5. For Neighborhood="Not assigned", make the value the same as Borough

In [9]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 6. Finally, print the number of rows of the cleaned dataframe

In [10]:
# shape of dataframe
toronto_df_grouped.shape

(17, 3)

### 7. Load the coordinates from the csv file on Coursera

In [11]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 8. Merge two tables to get the coordinates

In [13]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 9. Map the latitude and longitude on maps

In [14]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


### 10. Use the Foursquare API to explore the neighborhoods

In [15]:
CLIENT_ID = 'KDCT4XDGGZLR4H04L2JWP355LPPEEI3SEQSD3BRKPDDLIW55' # your Foursquare ID
CLIENT_SECRET = 'QFJU1DJ4FRLRGCR0HNAWN151LPYX02MDIQL3A0SNKPNDACOS' # your Foursquare Secret
VERSION = '20210730' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KDCT4XDGGZLR4H04L2JWP355LPPEEI3SEQSD3BRKPDDLIW55
CLIENT_SECRET:QFJU1DJ4FRLRGCR0HNAWN151LPYX02MDIQL3A0SNKPNDACOS


#### checking top 100 areas in a radius of 1000 metres (1kms)

In [16]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
#for post in zip(toronto_df_new['PostalCode']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']

    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
    
venues 
    

[('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  'Images Salon & Spa',
  43.80228301948931,
  -79.19856472801668,
  'Spa'),
 ('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  "Harvey's",
  43.80002047583462,
  -79.19830662127322,
  'Restaurant'),
 ('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  'RBC Royal Bank',
  43.79878248056552,
  -79.19709031445504,
  'Bank'),
 ('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  "Wendy's",
  43.8020084,
  -79.1980797,
  'Fast Food Restaurant'),
 ('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  'Wendy’s',
  43.80744841934756,
  -79.19905558052072,
  'Fast Food Restaurant'),
 ('M1B',
  'Scarborough',
  'Malvern, Rouge',
  43.806686299999996,
  -79.19435340000001,
  'Caribbean Wave',
  43.798557859976256,
  -79.19577725412623,
  'Caribbean Restau

#### Prettying up

In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(383, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
3,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant


#### Venues by PostCode

In [18]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M1B,Scarborough,"Malvern, Rouge",15,15,15,15,15,15
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",5,5,5,5,5,5
M1E,Scarborough,"Guildwood, Morningside, West Hill",23,23,23,23,23,23
M1G,Scarborough,Woburn,9,9,9,9,9,9
M1H,Scarborough,Cedarbrae,31,31,31,31,31,31
M1J,Scarborough,Scarborough Village,12,12,12,12,12,12
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",26,26,26,26,26,26
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",28,28,28,28,28,28
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",11,11,11,11,11,11


#### Unique VenueCategories

In [19]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 109 uniques categories.


In [20]:
venues_df['VenueCategory'].unique()[:50]

array(['Spa', 'Restaurant', 'Bank', 'Fast Food Restaurant',
       'Caribbean Restaurant', 'Paper / Office Supplies Store',
       'Coffee Shop', 'Trail', 'Auto Workshop', 'Chinese Restaurant',
       'Supermarket', 'Bakery', 'Park', 'Italian Restaurant',
       'Burger Joint', 'Breakfast Spot', 'Playground',
       'Fried Chicken Joint', 'Liquor Store', 'Food & Drink Shop',
       'Pizza Place', 'Smoothie Shop', 'Beer Store', 'Pharmacy',
       'Sports Bar', 'Discount Store', 'Sandwich Place',
       'Greek Restaurant', 'Bus Line', 'Convenience Store',
       'Indian Restaurant', 'Mobile Phone Shop', 'Hakka Restaurant',
       'Thai Restaurant', 'Music Store', 'Athletics & Sports',
       'Gas Station', 'Yoga Studio', 'Grocery Store', 'Wings Joint',
       'Asian Restaurant', 'Sporting Goods Shop', 'Intersection',
       'Ice Cream Shop', 'Japanese Restaurant', 'Bowling Alley',
       'Department Store', 'Hockey Arena', 'Light Rail Station',
       'Auto Garage'], dtype=object)

## 11. Next - Analyze each area.

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(383, 112)


,PostalCode,Borough,Neighborhoods,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Badminton Court,Bakery,Bank,Beach,Beer Store,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Farm,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Grouping

In [22]:

toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(16, 112)


,PostalCode,Borough,Neighborhoods,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Badminton Court,Bakery,Bank,Beach,Beer Store,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Farm,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.0

#### top 10 venues for the postcode

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(16, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",Fast Food Restaurant,Trail,Park,Paper / Office Supplies Store,Restaurant,Spa,Caribbean Restaurant,Bank,Bakery,Supermarket
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",Breakfast Spot,Burger Joint,Italian Restaurant,Park,Playground,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",Pizza Place,Bank,Fast Food Restaurant,Restaurant,Greek Restaurant,Sports Bar,Bus Line,Discount Store,Smoothie Shop,Liquor Store
3,M1G,Scarborough,Woburn,Coffee Shop,Park,Indian Restaurant,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Mobile Phone Shop,Fish Market,Diner,Discount Store
4,M1H,Scarborough,Cedarbrae,Bakery,Indian Restaurant,Bank,Grocery Store,Gas Station,Coffee Shop,Burger Joint,Bus Line,Caribbean Restaurant,Chinese Restaurant
5,M1J,Scarborough,Scarborough Village,Ice Cream Shop,Convenience Store,Bowling Alley,Playground,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Grocery Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",Pizza Place,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Discount Store,Convenience Store,Burger Joint,Intersection,Grocery Store,Light Rail Station
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",Coffee Shop,Bakery,Intersection,Bus Line,Convenience Store,Pizza Place,Sandwich Place,Restaurant,Pub,Park
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",Pizza Place,Beach,Ice Cream Shop,Park,Auto Garage,Sports Bar,Restaurant,Fast Food Restaurant,Farm,Event Space
9,M1N,Scarborough,"Birch Cliff, Cliffside West",Park,Convenience Store,Farm,Café,Restaurant,General Entertainment,Skating Rink,Diner,Thai Restaurant,Gym


## 12. Cluster the areas

#### run k means clustering to cluster areas into 10 clusters.

In [24]:
kclusters = 10

#print(toronto_grouped)

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

#print(toronto_grouped_clustering)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', means.labels_)

print(neighborhoods_venues_sorted)

16
    Cluster Labels PostalCode      Borough  \
0                5        M1B  Scarborough   
1                1        M1C  Scarborough   
2                0        M1E  Scarborough   
3                3        M1G  Scarborough   
4                7        M1H  Scarborough   
5                8        M1J  Scarborough   
6                0        M1K  Scarborough   
7                7        M1L  Scarborough   
8                4        M1M  Scarborough   
9                6        M1N  Scarborough   
10               7        M1P  Scarborough   
11               9        M1R  Scarborough   
12               2        M1S  Scarborough   
13               0        M1T  Scarborough   
14               2        M1V  Scarborough   
15               2        M1W  Scarborough   

                                        Neighborhoods 1st Most Common Venue  \
0                                      Malvern, Rouge  Fast Food Restaurant   
1              Rouge Hill, Port Union, Highland Creek   

In [28]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels and sort
toronto_merged=toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
toronto_merged.sort_values(["Cluster Labels"], inplace=True)

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(17, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Bank,Fast Food Restaurant,Restaurant,Greek Restaurant,Sports Bar,Bus Line,Discount Store,Smoothie Shop,Liquor Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Pizza Place,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Discount Store,Convenience Store,Burger Joint,Intersection,Grocery Store,Light Rail Station
13,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,0.0,Fast Food Restaurant,Pharmacy,Bank,Coffee Shop,Sandwich Place,Intersection,Italian Restaurant,Fried Chicken Joint,Mexican Restaurant,Deli / Bodega
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Breakfast Spot,Burger Joint,Italian Restaurant,Park,Playground,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,2.0,Chinese Restaurant,Shopping Mall,Bakery,Pizza Place,Caribbean Restaurant,Pool Hall,Café,Pharmacy,Park,Optical Shop


#### Finally, let's visualize the resulting clusters

In [29]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters